# 🎧 NB02: Data Processing

In [1]:
# Importing the necessary libraries
from dotenv import load_dotenv
from functions import *
from bs4 import BeautifulSoup
from pprint import pprint
from auth import *
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import pandas as pd
import json
import sqlite3
import matplotlib.pyplot as plt
import re

In [2]:
# Defining the function as "access_token" for later use
access_token = get_token()

### Step 1: Cleaning and Manipulating Playlist Data 
- Reading the top hits playlist and "girly" pop playlist JSON files
- Normalizing the data and removing duplicate rows
- Creating a final combined dataframe of both playlist rows.

In [3]:
# Reading the JSON file and saving it as "data"
with open('../data/raw/combined_top_hits.json', 'r') as f:
    data = json.load(f)

In [4]:
# Processing a list of items from the JSON file, storing it in "tracks"
tracks = pd.DataFrame([
    {
        'Track Name': item.get('track', {}).get('name'),
        'Track ID': item.get('track', {}).get('id'),
        'Artists': ", ".join(artist['name'] for artist in item.get('track', {}).get('artists', []))
    }
    for item in data['items']
])

In [5]:
# Converting tracks into a pandas dataframe
top_hits = pd.DataFrame(tracks)

In [6]:
# Here, I am adding featured artists to the list of artist names and normalizing them, alongside getting the number of artists featured/contributing on a song
top_hits['Artists'] = (
    top_hits['Artists']
    .str.lower()
    .str.strip()
    .apply(combine_artists)  # Combine main and featured artists
    .str.replace('feat.', 'feat', case=False)
    .str.strip()
)

# Count unique artists and add as a new column
top_hits['Artist Count'] = top_hits['Artists'].apply(lambda x: len(set(x.split(','))))

In [7]:
# Seeing if there are any duplicates in the top_hits dataframe
duplicates = top_hits.duplicated()
print(duplicates.any()) 

True


In [8]:
# Printing duplicate rows
duplicate_rows = top_hits[top_hits.duplicated()]
print(duplicate_rows)

            Track Name                Track ID       Artists  Artist Count
184  Sign of the Times  5Ohxk2dO5COHF1krpoPigN  harry styles             1


In [9]:
# Cleaning up the duplicates
top_hits_clean = (
    top_hits.drop_duplicates(keep='first')
    .copy()
    .assign(
        Track_Name=lambda df: df['Track Name'].str.lower().str.strip().str.replace(r'[^\w\s]', '', regex=True),
        Artists=lambda df: df['Artists'].str.lower().str.strip().str.split(', ')
    )
    .explode('Artists')
    .assign(Artists=lambda df: df['Artists'].str.replace(', ', ' &').astype('category'))
)

# Save to CSV
top_hits_clean.to_csv('../data/processed/top_hits.csv', index=False)

# Load and inspect data
top_hits_data = pd.read_csv('../data/processed/top_hits.csv')
top_hits_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Track Name    347 non-null    object
 1   Track ID      347 non-null    object
 2   Artists       347 non-null    object
 3   Artist Count  347 non-null    int64 
 4   Track_Name    347 non-null    object
dtypes: int64(1), object(4)
memory usage: 13.7+ KB


#### Moving on to "girly pop music" playlist data

In [10]:
with open('../data/raw/combined_women_pop.json', 'r') as f:
    data = json.load(f)

In [11]:
tracks = pd.DataFrame([
    {
        'Track Name': item.get('track', {}).get('name'),
        'Track ID': item.get('track', {}).get('id'),
        'Artists': ", ".join(artist['name'] for artist in item.get('track', {}).get('artists', []))
    }
    for item in data['items']
])

In [12]:
women_pop = pd.DataFrame(tracks)

In [13]:
women_pop['Artists'] = (
    women_pop['Artists']
    .str.lower()
    .str.strip()
    .apply(combine_artists)  # Combine main and featured artists
    .str.replace('feat.', 'feat', case=False)
    .str.strip()
)

# Count unique artists and add as a new column
women_pop['Artist Count'] = women_pop['Artists'].apply(lambda x: len(set(x.split(','))))

In [14]:
duplicates = women_pop.duplicated()
print(duplicates.any()) 

True


In [15]:
duplicate_rows = women_pop[women_pop.duplicated()]
print(duplicate_rows)

      Track Name                Track ID    Artists  Artist Count
201  Bad Romance  0SiywuOBRcynK0uKGWdCnn  lady gaga             1


In [16]:
# Cleaning up the duplicates
women_pop_clean = (
    women_pop.drop_duplicates(keep='first')
    .copy()
    .assign(
        Track_Name=lambda df: df['Track Name'].str.lower().str.strip().str.replace(r'[^\w\s]', '', regex=True),
        Artists=lambda df: df['Artists'].str.lower().str.strip().str.split(', ')
    )
    .explode('Artists')
    .assign(Artists=lambda df: df['Artists'].str.replace(', ', ' &').astype('category'))
)

# Save to CSV
women_pop.to_csv('../data/processed/women_pop.csv', index=False)

# Load and inspect data
women_pop_data = pd.read_csv('../data/processed/women_pop.csv')
women_pop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Track Name    280 non-null    object
 1   Track ID      280 non-null    object
 2   Artists       280 non-null    object
 3   Artist Count  280 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 8.9+ KB


In [17]:
playlist_df = pd.concat([top_hits_data, women_pop_data], ignore_index=True)

playlist_df.to_csv('../data/processed/playlists.csv', index=False)

playlist_df = pd.read_csv('../data/processed/playlists.csv')

duplicates = playlist_df[playlist_df.duplicated()]

In [18]:
# Remove duplicates (keep the first occurrence)
playlists_clean = playlist_df.drop_duplicates()

playlists_clean.to_csv('../data/processed/clean_playlists.csv', index=False)

### Step 2: Retreiving top 50 artists
I will be retrieving the top 50 artists whose songs feature the most number of times in the combined playlist dataframe. 


In [19]:
clean_df = pd.read_csv("../data/processed/clean_playlists.csv")

# Select the 'artists' column (similar to SQL SELECT artists FROM playlists)
artists_df = clean_df[['Artists']]

In [20]:
# Get the frequency of each artist
artist_counts = artists_df['Artists'].value_counts()
top_50_artists = artist_counts.head(50)

In [21]:
top_50_artists.to_csv('../data/processed/top_50_artists.csv', header=True)

Looking at the dataframe, these are the artists that I have chosen to include in my analysis:

| Male Artists | Female Arists |
| :--: | :--: |
| Justin Bieber | Rihanna |
| Bruno Mars | Ariana Grande |
| Ed Sheeran | Taylor Swift |
| Flo Rida | Nicki Minaj |
| Pharrell Williams | Britney Spears |

I chose these artists based on:
1) Gender - I wanted 5 male and 5 female artists
2) Pop artist status - To avoid overcomplicating and interfering the data analysis process, I decided to only choose pop artists (including Flo Rida and Nicki Minaj, who are also rap artists but constantly contribute to/feature on pop tracks)
3) Relevancy - I chose artists that I think are relevant, popular, and make or have made great contributions to defining 2000s/modern pop music

### Step 3: Retrieving top tracks and their lyrics

In [22]:
artists_info = {
    "Justin Bieber": "1uNFoZAHBGtllmzznpCI3s",
    "Bruno Mars": "0du5cEVh5yTK9QJze8zA0C",
    "Ed Sheeran": "6eUKZXaKkcviH0Ku9w2n3V",
    "Flo Rida": "0jnsk9HBra6NMjO2oANoPY",
    "Pharrell Williams": "2RdwBSPQiwcmiDo9kixcl8",
    "Rihanna": "5pKCCKE2ajJHZ9KAiaK11H",
    "Ariana Grande": "66CXWjxzNUsdJxJ2JdwvnR",
    "Taylor Swift": "06HL4z0CvFAxyc27GXpf02",
    "Nicki Minaj": "0hCNtLu0JehylgoiP8L4Gh",
    "Britney Spears": "26dSoYclwsYLMAKD3tpOr4"
}

In [23]:
top_tracks_df = pd.concat(
    [pd.DataFrame(get_top_tracks(artist_id, access_token)['tracks']).assign(artist=artist_name)
     for artist_name, artist_id in artists_info.items()],
    ignore_index=True
)

In [24]:
# Step 2: Select only relevant columns (create a copy to ensure no warning)
combined_tracks = top_tracks_df[['name', 'artist']].copy()

# Step 3: Add gender mapping
gender_mapping = {
    'Justin Bieber': 0,  # Male
    'Bruno Mars': 0,
    'Ed Sheeran': 0,
    'Flo Rida': 0,
    'Pharrell Williams': 0,
    'Rihanna': 1,        # Female
    'Ariana Grande': 1,
    'Taylor Swift': 1,
    'Nicki Minaj': 1,
    'Britney Spears': 1
}

combined_tracks.loc[:, 'gender'] = combined_tracks['artist'].map(gender_mapping)

combined_tracks.to_csv('../data/processed/combined_top_tracks.csv', index=False)

combined_tracks = pd.read_csv('../data/processed/combined_top_tracks.csv')

In [25]:
# Step 3: Apply the function to the DataFrame to fetch lyrics for each song
combined_tracks['lyrics'] = combined_tracks.apply(fetch_lyrics, axis=1)

# Step 4: Save the updated DataFrame with lyrics back to a new CSV
combined_tracks.to_csv('../data/raw/lyrics.csv', index=False)

Searching for "STAY (with Justin Bieber)" by Justin Bieber...
Done.
Searching for "Ghost" by Justin Bieber...
Done.
Searching for "Love Yourself" by Justin Bieber...
Searching for "Sorry" by Justin Bieber...
Done.
Searching for "Beauty And A Beat" by Justin Bieber...
Done.
Searching for "Mistletoe" by Justin Bieber...
Done.
Searching for "Baby" by Justin Bieber...
Done.
Searching for "bad guy" by Justin Bieber...
Done.
Searching for "I Don't Care (with Justin Bieber)" by Justin Bieber...
Done.
Searching for "What Do You Mean?" by Justin Bieber...
Done.
Searching for "Die With A Smile" by Bruno Mars...
Done.
Searching for "APT." by Bruno Mars...
Done.
Searching for "Locked out of Heaven" by Bruno Mars...
Done.
Searching for "Just the Way You Are" by Bruno Mars...
Searching for "That's What I Like" by Bruno Mars...
Done.
Searching for "When I Was Your Man" by Bruno Mars...
Done.
Searching for "Grenade" by Bruno Mars...
Done.
Searching for "It Will Rain" by Bruno Mars...
Done.
Searching f

In [26]:
lyrics = pd.read_csv('../data/raw/lyrics.csv')

# Step 2: Use regex to remove text between "embed" and "lyrics", including the words "embed" and "lyrics"
lyrics['lyrics'] = lyrics['lyrics'].str.replace(r'embed.*?lyrics', '', regex=True)

# Step 3: Save the updated DataFrame to a new CSV
lyrics.to_csv('../data/processed/cleaned_lyrics.csv', index=False)

In [27]:
cleaned_lyrics = pd.read_csv('../data/processed/cleaned_lyrics.csv')
cleaned_lyrics = pd.DataFrame(cleaned_lyrics)

In [28]:
cleaned_lyrics['lyrics'] = cleaned_lyrics['lyrics'].str.replace(
    r'ContributorsTranslations.*?Lyrics', 'Lyrics', regex=True
)

# Step 3: Save the updated DataFrame to a new CSV
cleaned_lyrics.to_csv('../data/processed/final_lyrics.csv', index=False)

# Reload the cleaned CSV for verification
final_lyrics = pd.read_csv('../data/processed/final_lyrics.csv')

In [29]:
error_rows = final_lyrics[final_lyrics['lyrics'].str.contains('Error fetching lyrics|not found', na=False)]

In [30]:
final_lyrics['lyrics'] = final_lyrics.apply(
    lambda row: row['lyrics'] if pd.notnull(row['lyrics']) else safe_get_song_lyrics(row),
    axis=1
)
# Save the updated DataFrame back to the CSV
final_lyrics.to_csv('../data/processed/final_lyrics_updated.csv', index=False)

In [31]:
error_rows = final_lyrics[final_lyrics['lyrics'].str.contains('Error fetching lyrics|not found', na=False)]

In [32]:
happy = get_song_lyrics_with_variations("Happy", "Pharrell Williams")

final_lyrics_updated = pd.read_csv('../data/processed/final_lyrics_updated.csv')

# Check if the song exists in the DataFrame
row_index = final_lyrics_updated[
    (final_lyrics_updated['name'] == "Happy") & 
    (final_lyrics_updated['artist'] == "Pharrell Williams")
].index

# If the song is found, update its lyrics
if not row_index.empty:
    final_lyrics_updated.at[row_index[0], 'lyrics'] = lyrics
    print(f"Lyrics updated for: {"Happy"} by {"Pharrell Williams"}")
else:
    print(f"Song '{"Happy"}' by {"Pharrell Williams"} not found in the DataFrame.")

# Save the updated DataFrame
final_lyrics_updated.to_csv('../data/processed/final_lyrics_updated.csv', index=False)

Searching for: Happy
Searching for "Happy" by Pharrell Williams...
Done.
Found lyrics for: Happy
Song 'Happy' by Pharrell Williams not found in the DataFrame.


Here, I am manually looking at the rows where the lyrics were not available (not an error but an issue of availability). In the next chunk, I'll be removing these

In [33]:
# Read the CSV and refetch lyrics in one step
updated_lyrics_df = refetch_lyrics_for_top_tracks(
    pd.read_csv('../data/processed/final_lyrics_updated.csv'), 
    indices_to_refetch
)

# Save the updated DataFrame back to a CSV file
updated_lyrics_df.to_csv("../data/processed/final_lyrics_final.csv", index=False)

# Reload the updated DataFrame (if further processing is required)
lyrics = updated_lyrics_df.copy()

Searching for "Swalla Nicki Minaj" by Nicki Minaj...
Done.
Searching for "Super Nicki Minaj" by Nicki Minaj...
Done.
Searching for "I Taylor Swift" by Taylor Swift...
Done.
Searching for "Dangerous Ariana Grande" by Ariana Grande...
Done.
Searching for "Save Ariana Grande" by Ariana Grande...
Done.
Searching for "Cash Pharrell Williams" by Pharrell Williams...
Done.
Searching for "Get Pharrell Williams" by Pharrell Williams...
Done.
Searching for "Get Pharrell Williams" by Pharrell Williams...
Done.
Searching for "My Flo Rida" by Flo Rida...
Done.
Searching for "Right Flo Rida" by Flo Rida...
Done.
Searching for "Bad Ed Sheeran" by Ed Sheeran...
Done.
Searching for "I Justin Bieber" by Justin Bieber...
Done.
Searching for "STAY Justin Bieber" by Justin Bieber...
Done.


In [ ]:
# Renaming the columns in my lyrics dataframe
# Matching those in the playlists dataframe, as they are more readable/explanatory
lyrics.rename(columns={
    'name': 'Track Name',
    'artist': 'Artists',
    'gender': 'Gender',
    'lyrics': 'Lyrics'
}, inplace=True)

# Saving the updated dataframe back to the CSV
lyrics.to_csv('../data/processed/final_lyrics_final.csv', index=False)

In [35]:
# Preprocess lyrics and save directly to a CSV file
lyrics['Lyrics'] = lyrics['Lyrics'].apply(preprocess_lyrics)
lyrics.to_csv('../data/processed/final_lyrics_final.csv', index=False)

# Reload the updated DataFrame if further processing is required
lyrics_df = pd.read_csv('../data/processed/final_lyrics_final.csv')

In [ ]:
# Loading the dataframes from the CSV files
clean_playlists_df = pd.read_csv('../data/processed/clean_playlists.csv')
final_lyrics_final_df = pd.read_csv('../data/processed/final_lyrics_final.csv')

# Cleaning both 'Track Name' and 'Artists' columns in both dataframes
clean_playlists_df['Track Name'] = clean_playlists_df['Track Name'].str.strip().str.lower()
clean_playlists_df['Artists'] = clean_playlists_df['Artists'].str.strip().str.lower()

final_lyrics_final_df['Track Name'] = final_lyrics_final_df['Track Name'].str.strip().str.lower()
final_lyrics_final_df['Artists'] = final_lyrics_final_df['Artists'].str.strip().str.lower()

# Connecting to and creating the SQLite database
conn = sqlite3.connect('../data/spotify.db')
cursor = conn.cursor()

# Enabling foreign key support, to later define Artists as the foreign key between the two tables
cursor.execute("PRAGMA foreign_keys = ON;")

# Dropping tables if they exist to ensure clean setup
cursor.execute("DROP TABLE IF EXISTS playlists;")
cursor.execute("DROP TABLE IF EXISTS top_tracks;")

# Creating the 'playlists' table with 'Artists' as a primary key
cursor.execute("""
CREATE TABLE IF NOT EXISTS playlists (
    "Track Name" TEXT NOT NULL,
    "Track ID" TEXT,
    "Artists" TEXT NOT NULL,
    "Artist Count" INTEGER,
    PRIMARY KEY ("Artists")
);
""")

# Inserting data from 'clean_playlists_df' into the 'playlists' table
cursor.executemany("""
INSERT OR REPLACE INTO playlists ("Track Name", "Track ID", "Artists", "Artist Count")
VALUES (?, ?, ?, ?)
""", clean_playlists_df[['Track Name', 'Track ID', 'Artists', 'Artist Count']].values)

# Creating the 'top_tracks' table with foreign key reference to 'Artists'
cursor.execute("""
CREATE TABLE IF NOT EXISTS top_tracks (
    "Track Name" TEXT NOT NULL,
    "Artists" TEXT NOT NULL,
    "Gender" TEXT,
    "Lyrics" TEXT,
    FOREIGN KEY ("Artists") REFERENCES playlists("Artists")
);
""")

# Fetch artists from the playlists table
cursor.execute('SELECT "Artists" FROM playlists')
existing_artists = set(row[0] for row in cursor.fetchall())  # Create a set of existing artists

# Filter the final_lyrics_final_df DataFrame to include only matching artists
filtered_df = final_lyrics_final_df[final_lyrics_final_df['Artists'].isin(existing_artists)]

# Insert matching rows into the 'top_tracks' table using executemany
cursor.executemany("""
INSERT OR REPLACE INTO top_tracks ("Track Name", "Artists", "Gender", "Lyrics")
VALUES (?, ?, ?, ?)
""", filtered_df[['Track Name', 'Artists', 'Gender', 'Lyrics']].values)

# Check how many rows didn't match
no_match_count = len(final_lyrics_final_df) - len(filtered_df)
print(f"\nTotal number of 'No match found' entries: {no_match_count}")

# Committing the changes to the database
conn.commit()

# Checking how many matches were missing
print(f"\nTotal number of 'No match found' entries: {no_match_count}")

# Verifying the data in the tables
cursor.execute("SELECT * FROM playlists LIMIT 5;")
print("Playlists Table Data:")
print(cursor.fetchall())

cursor.execute("SELECT * FROM top_tracks LIMIT 5;")
print("Top Tracks Table Data:")
print(cursor.fetchall())

# Closing the connection
conn.close()


Total number of 'No match found' entries: 0

Total number of 'No match found' entries: 0
Playlists Table Data:
[('glad you came', '5yDL13y5giogKs2fSNf7sj', 'the wanted', 1), ('dark horse', '5jrdCoLpJSvHHorevXBATy', 'juicy j', 2), ('closer', '7BKLCZ1jbUBVqRi2FVlTVw', 'halsey', 2), ('peaches (feat. daniel caesar & giveon)', '4iJyoBOLtHqaGxP12qzhQI', 'daniel caesar', 3), ('peaches (feat. daniel caesar & giveon)', '4iJyoBOLtHqaGxP12qzhQI', 'giveon', 3)]
Top Tracks Table Data:
[('stay (with justin bieber)', 'justin bieber', '0', " contributorsi do the same thing i told you that i never would\ni told you i'd change even when i knew i never could\ni know that i cant find nobody else as good as you\ni need you to stay need you to stay hey oh\n\n\ni get drunk wake up i'm wasted still\ni realize the time that i wasted here\ni feel like you can't feel the way i feel\noh ill be fucked up if you can't be right here\n\n\noh oohwoah oh oohwoah oohwoah\noh oohwoah oh oohwoah oohwoah\noh oohwoah oh oo

Click [here](https://github.com/lse-ds105/w10-summative-deyavuz/tree/main?tab=readme-ov-file#table-of-contents) to navigate back to the Table of Contents!